In [1]:
!pip --version

# To install pytorch -------------------------------------------------------------------------------------
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# To install Tensorflow ----------------------------------------------------------------------------------
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# To install Flax ----------------------------------------------------------------------------------------
!pip install flax

# or to install the latest version of Flax:
!pip install --upgrade git+https://github.com/google/flax.git

# Set up transformers
!pip install git+https://github.com/huggingface/transformers

# Check if installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

!pip install transformers[torch]

!pip install accelerate -U

!pip install datasets

!pip install opacus

# !pip install git+https://github.com/lxuechen/private-transformers.git

!pip install evaluate

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-w64klynl
  Running command git clone --filter=blob:none --quiet https://github.com/google/flax.git /tmp/pip-req-build-w64klynl
  Resolved https://github.com/google/flax.git to commit 85dfad242e56098849dbf05e7e4657b3a40820f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for flax: filename=flax-0.8.0-py3-none-any.whl size=635291 sha256=0941195ff0a2c0da85d2289c743fe48d738a3d6c7f7fb205b8ec345e42bae2ce
  Stored in directory: /tmp/pi

In [2]:
from datasets.download.streaming_download_manager import xexists
from datasets import Dataset
# Libraries to import
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import BinaryCrossentropy
from datasets import load_metric
import torch
import numpy as np
from transformers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pandas as pd
# from private_transformers import PrivacyEngine
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import evaluate
from evaluate import evaluator
import opacus
from opacus import PrivacyEngine

In [3]:
# Preprocessing
dataset = load_dataset("glue", "qqp")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2) # as output 0 or 1
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 390965
    })
})

In [4]:
# Assuming the dataset has a 'train' split, modify this according to your dataset's splits
data_split = dataset['train']

# Convert the dataset split to a pandas DataFrame for easier splitting
df = data_split.to_pandas()
df = df[0:150000]

# Split the dataset into train and test sets using train_test_split from sklearn
# df1, df2 = train_test_split(df, test_size = 0.5, random_state = 42)
train_df, test_df = train_test_split(df , test_size=0.2, random_state=42)

# Convert the splits back to datasets
train_dataset = train_df.reset_index(drop=True)
test_dataset = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

x_train1 = list(train_dataset["question1"])
x_train2 = list(train_dataset["question2"])
y_train = list(train_dataset["label"])

x_test1 = list(test_dataset["question1"])
x_test2 = list(test_dataset["question2"])
y_test = list(test_dataset["label"])

X_train_tokenized = tokenizer(x_train1, x_train2, padding=True, truncation=True, max_length = 128)
# X_val_tokenized = tokenizer(x_validation, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(x_test1, x_test2, padding=True, truncation = True, max_length = 128)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to('cuda') for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).to('cuda')
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
# val_dataset = Dataset(X_val_tokenized, y_validation)
test_dataset = Dataset(X_test_tokenized, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 512, drop_last = True)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size = 512, drop_last = True)
# val_dataloader = DataLoader(val_dataset, batch_size = 500)

In [5]:
trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
total_params = 0
trainable_params = 0
l = []

for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

for layer in trainable_layers:
    for p in layer.parameters():
        l.append(p)
        p.requires_grad = True
        trainable_params += p.numel()

optimizer = AdamW(model.parameters(), lr=5e-5, eps = 1e-8)

print(f"Total parameters count: {total_params}") # ~108M
print(f"Trainable parameters count: {trainable_params}") # ~7M

Total parameters count: 4386178
Trainable parameters count: 215042


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
model.train()
privacy_engine = PrivacyEngine()
model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta= 1 / len(train_dataloader),
    target_epsilon= 8,
    epochs= 3,
    max_grad_norm = 0.1,
)

/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the smallest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


In [7]:
def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()
    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:

        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs[:2]

            log = logits.cpu()
            log = log.numpy()

            preds = np.argmax(log, axis=1)
            labels = batch['labels'].detach().cpu().numpy()

            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

model.train()

# Training loop
optimizer.zero_grad()  # Explicitly zero the gradient buffers

for epoch in range(20):  # Number of epochs
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim = -1)

    eval_loss, eval_accuracy = evaluate(model)
    print("eval loss",eval_loss)
    print("accuracy: ",eval_accuracy)
    print("ends")

    model.train()  # Set the model back to training mode


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1324: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "


eval loss 1.5931633279241364
accuracy:  0.6290072737068966
ends
eval loss 2.118783776102395
accuracy:  0.6290072737068966
ends
eval loss 2.137994009872963
accuracy:  0.6290072737068966
ends
eval loss 2.1007401511586945
accuracy:  0.6290072737068966
ends
eval loss 2.012317560870072
accuracy:  0.6290072737068966
ends
eval loss 1.920884103610598
accuracy:  0.6290072737068966
ends
eval loss 1.8082526511159436
accuracy:  0.6290072737068966
ends
eval loss 1.509137881213221
accuracy:  0.6539938038793104
ends
eval loss 1.4987187673305642
accuracy:  0.6826508620689655
ends
eval loss 1.6453356557878955
accuracy:  0.6885102370689655
ends
eval loss 1.7123225331306458
accuracy:  0.6872306034482759
ends
eval loss 1.666950153893438
accuracy:  0.6928879310344828
ends
eval loss 1.6939132624659046
accuracy:  0.6915409482758621
ends
eval loss 1.698976467395651
accuracy:  0.692315463362069
ends
eval loss 1.6956490524883927
accuracy:  0.6946053340517241
ends
eval loss 1.673918210226914
accuracy:  0.6971646